In [208]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import Resize
from sklearn.feature_extraction.text import TfidfVectorizer


In [209]:
train_data = []
with open('数据\\train.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
    train_data = []
    for k in range(len(lines)-1):
        train_data.append([])
    for k in range(1,len(lines)):
        guid,label = lines[k].split(',')
        train_data[k-1].append(guid)
        if label=='positive\n':
            label = 0
        elif label=='neutral\n':
            label = 1
        elif label=='negative\n':
            label = 2
        train_data[k-1].append(label)

train = []
for k in range(len(train_data)):
    train.append(int(train_data[k][0]))
    
files = os.listdir('数据\data')           
for k in range(len(train_data)):
    txt_path = train_data[k][0] + '.txt'
    txt_path = os.path.join('数据\data',txt_path)
    with open(txt_path,'rb') as f:
        txt = f.read()
        wash_txt = txt.decode("utf-8", "replace").replace('&', '').replace('#', 'tag ').replace('!', ' exclamationmark').replace('?', ' questionmark').replace('^0^', ' ').replace(',', ' ').replace('-', ' ').replace('\r\n',' ')
        train_data[k].append(wash_txt)
    img_path = train_data[k][0] + '.jpg'
    img_path = os.path.join('数据\data',img_path)
    resize_transform = Resize((224, 224))
    img = Image.open(img_path).convert('RGB')
    resized_img = resize_transform(img)
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
    tensor_img = transform(resized_img)
    train_data[k].append(tensor_img)          #编号、情感标签、文本、图片tensor


In [210]:
with open('数据\\test_without_label.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
    test = []
    for k in range(1,len(lines)):
        guid,_ = lines[k].split(',')
        test.append(guid)                    #511个用于测试
            
test_data = []
for k in range(len(test)):
    test_data.append([])
for k in range(len(test)):
    txt_path = str(test[k]) + '.txt'
    txt_path = os.path.join('数据\data',txt_path)
    with open(txt_path,'rb') as f:
        txt = f.read()
        test_data[k].append(test[k])
        test_data[k].append('3')
        wash_txt = txt.decode("utf-8", "replace").replace('&', '').replace('#', 'tag ').replace('!', ' exclamationmark').replace('?', ' questionmark').replace('^0^', ' ').replace(',', ' ').replace('-', ' ').replace('\r\n',' ')
        test_data[k].append(wash_txt)
    img_path = str(test[k]) + '.jpg'
    img_path = os.path.join('数据\data',img_path)
    resize_transform = Resize((224, 224))
    img = Image.open(img_path).convert('RGB')
    resized_img = resize_transform(img)
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
    tensor_img = transform(resized_img)
    test_data[k].append(tensor_img) #编号、null标签、文本、图片tensor

In [212]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer()
train_txt = [data[2] for data in train_data]
test_txt = [data[2] for data in test_data]


train_txt_encoded = vectorizer.fit_transform(train_txt).toarray()
test_txt_encoded = vectorizer.transform(test_txt).toarray()

for i in range(len(train_data)):
    train_data[i][2] = torch.tensor(train_txt_encoded[i])

for i in range(len(test_data)):
    test_data[i][2] = torch.tensor(test_txt_encoded[i])

#print(test_data[0][2])  # torch.Size([15709])

In [213]:
#print(train_data[0][2])
train_data,val_data = train_test_split(train_data,test_size=0.1)
#print(len(train_data)) #3600
#print(len(val_data)) #400

3600
400


In [214]:
class txt_stream(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    def forward(self, x):
        #print(x.shape)
        x = self.fc(x)
        #print(x.shape)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class img_stream(nn.Module):
    def __init__(self,input_channels, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32*56*56, output_size)
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

class TwoStream(nn.Module): 
    def __init__(self, txt_input_size, txt_hidden_size, img_input_channels, img_output_size, label_classes):
        super().__init__()
        self.txt_stream = txt_stream(txt_input_size, txt_hidden_size, label_classes)
        self.img_stream = img_stream(img_input_channels, img_output_size)
        self.fc = nn.Linear(txt_hidden_size + img_output_size, label_classes)
    def forward(self, txt_inputs, img_inputs):
        txt_out = self.txt_stream(txt_inputs)
        img_out = self.img_stream(img_inputs)
        img_out = img_out.unsqueeze(1)
        img_out = img_out.squeeze(dim=1)
        combined_out = torch.cat((txt_out, img_out), dim=-1)
        out = self.fc(combined_out) 
        return out

In [215]:
txt_input_size = 15709
txt_hidden_size = 3
img_input_channels = 3
img_output_size = 3
label_classes = 3
epochs = 30
num_classes = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
txt_model = txt_stream(txt_input_size, txt_hidden_size, num_classes)
img_model = img_stream(img_input_channels, img_output_size)
two_model = TwoStream(txt_input_size, txt_hidden_size, img_input_channels, img_output_size, label_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(two_model.parameters(), lr=0.01)
encoder = preprocessing.LabelEncoder() 

# 准备训练数据
train1 = []
train2 = []
train3 = []
for k in range(len(train_data)):
    train1.append(train_data[k][1:])
    train2.append(train_data[k][1:3])
    train3.append([train_data[k][1],train_data[k][3]])
#print(train[0])
two_train_batch = torch.utils.data.DataLoader(train1,batch_size=64, shuffle=True)
txt_train_batch = torch.utils.data.DataLoader(train2,batch_size=64, shuffle=True)
img_train_batch = torch.utils.data.DataLoader(train3,batch_size=64, shuffle=True)

val1 = []
val2 = []
val3 = []
for k in range(len(val_data)):
    val1.append(val_data[k][1:])
    val2.append(val_data[k][1:3])
    val3.append([val_data[k][1],val_data[k][3]])
#print(train[0])
two_val_batch = torch.utils.data.DataLoader(val1,batch_size=64, shuffle=True)
txt_val_batch = torch.utils.data.DataLoader(val2,batch_size=64, shuffle=True)
img_val_batch = torch.utils.data.DataLoader(val3,batch_size=64, shuffle=True)



In [216]:
def two_model_train(model,train_batch,val_batch):
    best_acc = 0
    for epoch in range(epochs):
        train_loss = 0.0
        model.train()
        for i,(label,txt_train,img_train) in enumerate(train_batch):
            optimizer.zero_grad() 
            txt_train, img_train, label = txt_train.to(device), img_train.to(device), label.to(device)
            txt_train = txt_train.to(torch.float32)  # 转换为单精度浮点数
            img_train = img_train.to(torch.float32)
            outputs = model(txt_train, img_train).squeeze()
            loss = F.cross_entropy(outputs, label)    
            train_loss = train_loss + loss.item() * txt_train.size(0)
            loss.backward()
            optimizer.step()
        train_loss = train_loss / len(train_batch.dataset)
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for i,(label,txt_val,img_val) in enumerate(val_batch):
                txt_val, img_val, label = txt_val.to(device), img_val.to(device), label.to(device)
                txt_val = txt_val.to(torch.float32)
                img_val = img_val.to(torch.float32)  
                outputs = model(txt_val, img_val).squeeze()
                _, pred = torch.max(outputs.data,dim=1)
                val_total = val_total + label.size(0)
                print(pred)
                val_correct = val_correct + (pred == label).sum().item()
                print(val_correct)
            val_acc = val_correct / val_total
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), './best_two_model.pt')
                print('模型进行了更新！')
        print('迭代轮次 [{}/{}], 样本平均训练损失: {:.4f}, 验证准确率: {:.4f}'.format(epoch+1, epochs, train_loss,val_acc))

In [217]:
def one_model_train(model,train_batch,val_batch):
    best_acc = 0
    for epoch in range(epochs):
        train_loss = 0.0
        model.train()
        for i,(label,what_train) in enumerate(train_batch):
            optimizer.zero_grad() 
            what_train, label = what_train.to(device),label.to(device)
            what_train = what_train.to(torch.float32)
            outputs = model(what_train).squeeze()
            loss = F.cross_entropy(outputs, label)    
            train_loss = train_loss + loss.item() * what_train.size(0)
            loss.backward()
            optimizer.step()
        train_loss = train_loss / len(train_batch.dataset)
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for i,(label,what_val) in enumerate(val_batch):
                what_val, label = what_val.to(device),label.to(device)
                what_val = what_val.to(torch.float32)
                outputs = model(what_val).squeeze()
                _, pred = torch.max(outputs.data,dim=1)
                val_total = val_total + label.size(0)
                print(pred)
                val_correct = val_correct + (pred == label).sum().item()
                print(val_correct)
            val_acc = val_correct / val_total
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), './best_one_model.pt')
                print('模型进行了更新！')
        print('迭代轮次 [{}/{}], 样本平均训练损失: {:.4f}, 验证准确率: {:.4f}'.format(epoch+1, epochs, train_loss,val_acc))

In [218]:
one_model_train(txt_model,txt_train_batch,txt_val_batch)

tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
20
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
37
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
54
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
76
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [219]:
one_model_train(img_model,img_train_batch,img_val_batch)

tensor([0, 0, 1, 1, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 1, 0, 1, 1, 0, 0, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
14
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 2, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1,
        1, 1, 0, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 0, 1])
26
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 2,
        1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1])
37
tensor([1, 1, 2, 0, 2, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 2,
        1, 2, 1, 0, 1, 2, 1, 0, 1, 2, 2, 2, 1, 1, 1, 0])
47
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 2, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [224]:
two_model_train(two_model,two_train_batch,two_val_batch)

tensor([0, 0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0, 0, 2, 0, 0,
        0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0])
34
tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0,
        2, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2,
        0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0])
67
tensor([1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0,
        2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0])
97
tensor([1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0])
127
tensor([2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 1, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0

In [221]:
test1 = []
test2 = []
test3 = []
for k in range(len(test_data)):
    test1.append(test_data[k][2:])
    test2.append(test_data[k][2])
    test3.append(test_data[k][3])
two_test_batch = torch.utils.data.DataLoader(test1,batch_size=64, shuffle=True)
txt_test_batch = torch.utils.data.DataLoader(test2,batch_size=64, shuffle=True)
img_test_batch = torch.utils.data.DataLoader(test3,batch_size=64, shuffle=True)

In [226]:
#'positive-0':'neutral-1':'negative-2'
best_model = TwoStream(txt_input_size, txt_hidden_size, img_input_channels, img_output_size, label_classes)
best_model.load_state_dict(torch.load('./best_two_model.pt'))
best_model = best_model.to(device)
best_model.eval()
test_correct = 0
with torch.no_grad():
    predall = []
    for i,(txt_test, img_test) in enumerate(two_test_batch):
        txt_test, img_test = txt_test.to(device), img_test.to(device)
        txt_test = txt_test.to(torch.float32)  # 转换为单精度浮点数
        img_test = img_test.to(torch.float32)  
        outputs = best_model(txt_test,img_test).squeeze()
        _, pred = torch.max(outputs.data, 1)
        print(pred)
        for p in pred:
            predall.append(int(p))

tensor([2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0,
        0, 2, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0,

In [228]:
with open('test_without_label.txt','r+') as f:
    lines = f.readlines()
    for i in range(1,len(predall)+1):
        line_parts = lines[i].split(',')
        if (predall[i-1]==0):
            line_parts[1] = 'positive' + '\n'
            lines[i] = ','.join(line_parts)
        elif(predall[i-1]==1):
            line_parts[1] = 'neutral' + '\n'
            lines[i] = ','.join(line_parts)
        elif(predall[i-1]==2):
            line_parts[1] = 'negative' + '\n'
            lines[i] = ','.join(line_parts)
        # 回到文件开头，清空文件内容
        f.seek(0)
        f.truncate()
        # 写入修改后的内容
        f.writelines(lines)